In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image data generator for training
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the images to [0, 1]
    rotation_range=40,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Random shear
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'  # Fill mode for new pixels
)

# Define the validation data generator (without augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

# Define the train and validation directories
train_dir = 'C:/Users/ksaks/Downloads/DATA/Training/train'  # Directory containing 'real' and 'deepfake' folders
val_dir = r'C:\Users\ksaks\Downloads\DATA\Training\VAL'   # Directory containing 'real' and 'deepfake' folders


# Create generators for training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # Resize images to 299x299 for InceptionV3
    batch_size=32,
    class_mode='binary'  # Binary labels (0 or 1)
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)


Found 4812 images belonging to 2 classes.
Found 2068 images belonging to 2 classes.


In [3]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models

# Load the pre-trained InceptionV3 model (exclude the top layers)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))


# Add custom layers on top for binary classification
model = models.Sequential([
    base_model,  # The pre-trained InceptionV3 model
    layers.GlobalAveragePooling2D(),  # Global average pooling layer
    layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
pip uninstall tensorflow

In [ ]:
pip install tensorflow==2.15.0

In [1]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [4]:
import os
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

# Callback to save the top 10 models based on validation accuracy
class TopKModelSaver(Callback):
    def __init__(self, k=30, monitor='val_accuracy', mode='max', save_dir='./'):
        super(TopKModelSaver, self).__init__()
        self.k = k  # Number of top models to keep
        self.monitor = monitor  # Metric to monitor
        self.mode = mode  # 'max' for maximizing the metric, 'min' for minimizing
        self.best_models = []  # Store the top models (metric, file path)
        self.save_dir = save_dir  # Directory to save models

        # Define the comparison function based on mode
        if mode == 'max':
            self.comparator = lambda x, y: x > y
        elif mode == 'min':
            self.comparator = lambda x, y: x < y
        else:
            raise ValueError("Mode must be 'max' or 'min'.")

    def on_epoch_end(self, epoch, logs=None):
        # Get the monitored metric value
        metric_value = logs.get(self.monitor)
        if metric_value is None:
            print(f"Metric {self.monitor} not found in logs. Skipping...")
            return

        # Save the model for this epoch in .h5 format
        model_path = os.path.join(self.save_dir, f"model_epoch_{epoch + 1:02d}_{self.monitor}_{metric_value:.4f}.h5")
        self.model.save(model_path)

        # Add the model to the best models list
        self.best_models.append((metric_value, model_path))

        # Sort and retain only the top k models
        self.best_models.sort(key=lambda x: x[0], reverse=(self.mode == 'max'))
        if len(self.best_models) > self.k:
            # Remove the worst model from disk
            worst_model = self.best_models.pop(-1)
            if os.path.exists(worst_model[1]):
                os.remove(worst_model[1])
                print(f"Removed model: {worst_model[1]}")

# Callback to save the last model
class SaveLastModelCallback(Callback):
    def __init__(self, save_path):
        super(SaveLastModelCallback, self).__init__()
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        # Save the current model as the "last model" in .h5 format
        self.model.save(self.save_path)
        print(f"Saved the last model to {self.save_path}")

# Directory for saving models
save_dir = "C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/"
# Initialize the TopKModelSaver callback
top_k_saver = TopKModelSaver(
    k=30,
    monitor='val_loss',
    mode='max',
    save_dir=save_dir
)

# Initialize the SaveLastModelCallback
last_model_callback = SaveLastModelCallback(
    save_path=os.path.join(save_dir, 'Inception_net_last_model.h5')  # Save as .h5
)

# Set up the ModelCheckpoint callback (save in .h5 format)
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(save_dir, 'Inception_net_epoch_{epoch:02d}_loss_{val_loss:.4f}.h5'),  # Use .h5 extension
    monitor='val_loss',
    save_best_only=False,  # Save every epoch, not just the best
    mode='max',
    verbose=1
)

# Train the model with all callbacks
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[checkpoint_callback, top_k_saver, last_model_callback]
)


Epoch 1/50


150/150 [==============================] - ETA: 0s - loss: 0.4678 - accuracy: 0.7910
Epoch 1: saving model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception\Inception_net_epoch_01_loss_0.5474.h5


C:\Users\ksaks\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saved the last model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/Inception_net_last_model.h5
150/150 [==============================] - 263s 2s/step - loss: 0.4678 - accuracy: 0.7910 - val_loss: 0.5474 - val_accuracy: 0.7188
Epoch 2/50
150/150 [==============================] - ETA: 0s - loss: 0.3433 - accuracy: 0.8715
Epoch 2: saving model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception\Inception_net_epoch_02_loss_0.5442.h5
Saved the last model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/Inception_net_last_model.h5
150/150 [==============================] - 266s 2s/step - loss: 0.3433 - accuracy: 0.8715 - val_loss: 0.5442 - val_accuracy: 0.7222
Epoch 3/50
150/150 [==============================] - ETA: 0s - loss: 0.3018 - accuracy: 0.8916
Epoch 3: saving model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception\Inception_net_epoch_03_loss_0.5601.h5
Saved the last model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/Inception_net_last_model.

Epoch 20/50
150/150 [==============================] - ETA: 0s - loss: 0.2422 - accuracy: 0.9096
Epoch 20: saving model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception\Inception_net_epoch_20_loss_0.5763.h5
Saved the last model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/Inception_net_last_model.h5
150/150 [==============================] - 236s 2s/step - loss: 0.2422 - accuracy: 0.9096 - val_loss: 0.5763 - val_accuracy: 0.7344
Epoch 21/50
150/150 [==============================] - ETA: 0s - loss: 0.2542 - accuracy: 0.9015
Epoch 21: saving model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception\Inception_net_epoch_21_loss_0.6173.h5
Saved the last model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/Inception_net_last_model.h5
150/150 [==============================] - 229s 2s/step - loss: 0.2542 - accuracy: 0.9015 - val_loss: 0.6173 - val_accuracy: 0.7480
Epoch 22/50
150/150 [==============================] - ETA: 0s - loss: 0.2425 - accuracy: 0.9056
E

Epoch 37/50
150/150 [==============================] - ETA: 0s - loss: 0.2322 - accuracy: 0.9090
Epoch 37: saving model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception\Inception_net_epoch_37_loss_0.5683.h5
Removed model: C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/model_epoch_37_val_loss_0.5683.h5
Saved the last model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/Inception_net_last_model.h5
150/150 [==============================] - 270s 2s/step - loss: 0.2322 - accuracy: 0.9090 - val_loss: 0.5683 - val_accuracy: 0.7363
Epoch 38/50
150/150 [==============================] - ETA: 0s - loss: 0.2327 - accuracy: 0.9098
Epoch 38: saving model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception\Inception_net_epoch_38_loss_0.5753.h5
Removed model: C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/model_epoch_33_val_loss_0.5738.h5
Saved the last model to C:/Users/ksaks/Downloads/DATA/Models/Model_Inception/Inception_net_last_model.h5
150/150 [=================